In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [4]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [4]:
train.isnull().sum()

ID                                  0
Park_ID                             0
Date                                0
Direction_Of_Wind                3931
Average_Breeze_Speed             3931
Max_Breeze_Speed                 3936
Min_Breeze_Speed                 3934
Var1                             8282
Average_Atmospheric_Pressure    40195
Max_Atmospheric_Pressure        40195
Min_Atmospheric_Pressure        40195
Min_Ambient_Pollution           31645
Max_Ambient_Pollution           31645
Average_Moisture_In_Park           40
Max_Moisture_In_Park               40
Min_Moisture_In_Park               40
Location_Type                       0
Footfall                            0
dtype: int64

In [5]:
def transform_data(data):
    data['Date'] = pd.to_datetime(data['Date'])
    data['Month'] = data['Date'].dt.month
    #data['Year'] = data['Date'].dt.year
    data['Day'] = data['Date'].dt.day
    # data['WeekOfYear'] = data['Date'].dt.weekofyear
    # data['DayOfYear'] = data['Date'].dt.dayofyear + 1
    data['WeekDay'] = data['Date'].dt.weekday
    
    # Average Ambient Pollution bhi nikal lo
    data['Diff_Breeze_Speed'] = data['Max_Breeze_Speed'] - data['Min_Breeze_Speed']
    data['Diff_Atmospheric_Pressure'] = data['Max_Atmospheric_Pressure'] - data['Min_Atmospheric_Pressure']
    data['Diff_Ambient_Pollution'] = data['Max_Ambient_Pollution'] - data['Min_Ambient_Pollution']
    data['Average_Ambient_Pollution'] = (data['Max_Ambient_Pollution'] + data['Min_Ambient_Pollution'])/2
    data['Diff_Moisture_In_Park'] = data['Max_Moisture_In_Park'] - data['Min_Moisture_In_Park']
    # data['Breeze_Speed'] = data['Diff_Breeze_Speed'] / data['Average_Breeze_Speed']
    # data['Atm_Pressure'] = data['Diff_Atmospheric_Pressure'] / data['Average_Atmospheric_Pressure']
    # data['Amb_Pollution'] = data['Diff_Ambient_Pollution'] / data['Average_Ambient_Pollution']
    # data['Moisture_In_Park'] = data['Diff_Moisture_In_Park'] / data['Average_Moisture_In_Park']
    #data = data.drop(labels=['Date', 'ID'], axis=1)
    data = data.drop(labels=['Date', 'ID','Max_Ambient_Pollution','Max_Atmospheric_Pressure','Max_Breeze_Speed' ,'Max_Moisture_In_Park', 'Min_Moisture_In_Park','Min_Ambient_Pollution', 'Min_Atmospheric_Pressure', 'Min_Breeze_Speed' ], axis=1)
    # data = data.drop(labels=['Diff_Breeze_Speed', 'Diff_Atmospheric_Pressure', 'Diff_Ambient_Pollution', 'Diff_Moisture_In_Park', 'Average_Moisture_In_Park', 'Average_Ambient_Pollution', 'Average_Atmospheric_Pressure', 'Average_Breeze_Speed'], axis =1)
    # data['Year'] = data['Year'] - data['Year'].min()
    columns = data.columns.values
    # data.loc[data['Direction_Of_Wind'].isnull(), 'Direction_Of_Wind'] = np.round(np.random.uniform(0,360))
    data['Direction_Of_Wind'].fillna(np.round(np.random.uniform(0,360)), inplace=True)
    # data = MICE().complete(data)
    
    for col in columns:
        # fill missing data with median values # GENERIC WAY
        #data.loc[data[col].isnull(), col] = data[col].median()
        # random sampling
        # data.loc[data[col].isnull(), col] = data.loc[data[col].notnull(), col].sample( n = 1 )
        data[col].fillna(np.random.choice(data.loc[data[col].notnull(), col]), inplace=True)
        # data[col].fillna(data[col].median(), inplace=True)
        # if (col not in['Park_ID', 'Location_Type', 'Footfall', 'Month', 'Year', 'Day', 'WeekDay', 'WeekOfYear', 'DayOfYear']):
            # data[col] = StandardScaler().fit_transform(data[col])
    
    return data;

In [29]:
df_train_data = transform_data(train)
df_test_data = transform_data(test)

df_train_labels = df_train_data['Footfall']
df_train_data = df_train_data.drop('Footfall', axis=1)

df_train_data = df_train_data.values
df_test_data = df_test_data.values
df_train_labels = df_train_labels.values
df_train_data = df_train_data[0:25000, :]
df_train_labels = df_train_labels[0:25000]

TypeError: unhashable type

In [30]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(15, input_dim=15, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [39]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=100, batch_size=40, verbose=1)

In [40]:
kfold = KFold(n=len(df_train_data), n_folds=4, random_state=seed)
results = cross_val_score(estimator, df_train_data, df_train_labels, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/100
18750/18750 [==============================] - 0s - loss: 246013.4312     
Epoch 2/100
18750/18750 [==============================] - 0s - loss: 53444.4740     
Epoch 3/100
18750/18750 [==============================] - 0s - loss: 52586.4591     
Epoch 4/100
18750/18750 [==============================] - 0s - loss: 51537.0003     
Epoch 5/100
18750/18750 [==============================] - 0s - loss: 50494.7393     
Epoch 6/100
18750/18750 [==============================] - 0s - loss: 49309.6603     
Epoch 7/100
18750/18750 [==============================] - 0s - loss: 48169.4137     
Epoch 8/100
18750/18750 [==============================] - 0s - loss: 47141.8493     
Epoch 9/100
18750/18750 [==============================] - 0s - loss: 46212.2336     
Epoch 10/100
18750/18750 [==============================] - 0s - loss: 45637.8862     
Epoch 11/100
18750/18750 [==============================] - 0s - loss: 45267.6075     
Epoch 12/100
18750/18750 [=========================

KeyboardInterrupt: 